In [41]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [42]:

path_data_dir = "/home/juan/Desktop/juan/bio/mirna_mite/data/res/mrcv_all/"
path_res_dir = "/home/juan/Desktop/juan/bio/mirna_mrcv/data/"
path_fasta_res = "/home/juan/Desktop/juan/bio/mirna_mrcv/data/mirna.deg.Y.fasta"
path_deg_file = path_res_dir + "mirna.deg.csv"

In [43]:
path_results = path_data_dir + "Results.txt"
df_res = pd.read_csv(path_results, sep="\t")
df_res.head(2)

,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,DicerCall,MIRNA,PhaseScore,Short,Long,20,21,22,23,24
0,1A:209-460,Cluster_1,252,118,0.926,64,1.0,+,CGCGGAUUGCUCGAGCUGCUCACGCGGCGAGAGCGGGUCG,25,...,N,N2,-1.0,28,71,2,1,6,6,4
1,1A:690-750,Cluster_2,61,78,0.612,0,1.0,+,UAGACCGACUUUGUGAAAUG,5,...,N,N2,-1.0,7,54,7,0,1,2,7


In [44]:
df_deg = pd.read_csv(path_deg_file, sep=",")
df_deg.head(2)

,Gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,X21dpiR1C.trimmed,X21dpiR3C.trimmed,X21dpiR1T.trimmed,X21dpiR3T.trimmed
0,Cluster_9889,298.598043,-1.471187,0.264059,-5.571425,2.526645e-08,0.000002,417.884204,459.336142,137.863421,179.308403
1,Cluster_35781,411.843895,-0.908261,0.236510,-3.840258,1.229050e-04,0.005899,583.212178,491.732645,306.089619,266.341137


In [45]:
len(df_deg[df_deg.log2FoldChange > 0].index), len(df_deg[df_deg.log2FoldChange < 0].index)

(4, 7)

In [46]:
df_deg = df_deg[['Gene','log2FoldChange']]

In [47]:
df_deg_y = pd.merge(df_res, df_deg, left_on='Name', right_on='Gene')
print(len(df_deg_y.index))
df_deg_y.head(2)

11


,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,PhaseScore,Short,Long,20,21,22,23,24,Gene,log2FoldChange
0,2A:485771022-485771154,Cluster_5906,133,5985,46.972,305,1.000,+,UAGGCGGUCACCUUGGCUAG,2883,...,-1.0,1080,3,4050,798,52,0,2,Cluster_5906,-0.915816
1,2A:652328872-652329011,Cluster_6309,140,1185,9.300,1,0.001,-,UGCCAAAGGAGAGUUGCCCUG,1136,...,-1.0,14,0,3,1155,11,2,0,Cluster_6309,-1.201318


In [48]:
buffer_seqs = []
for k,v in df_deg_y.iterrows():
    description = '%s l2fc:%f' % (v['#Locus'], v['log2FoldChange'])
    seq = SeqRecord(Seq(v.MajorRNA), id=v.Name, description=description)
    buffer_seqs.append(seq)
SeqIO.write(buffer_seqs, path_fasta_res, "fasta")
print(len(buffer_seqs))
print(path_fasta_res)

11
/home/juan/Desktop/juan/bio/mirna_mrcv/data/mirna.deg.Y.fasta


In [ ]:
df_deg_y.to_csv(path_res_dir + "mirna.res.deg.csv", sep="\t", index=None)